# INBOX

See all incoming messages here

In [20]:
#from s3i import IdentityProvider, Broker, UserMessage, TokenType, Key, Directory
import s3i
import time 
import base64 
import getpass
import receiver
import pykka

In [21]:
USERNAME = "gebhard_workshop" #None
PASSWORD = "baum" #getpass.getpass('[S3I]: Please enter your password:')

In [22]:
CLIENT = "s3i:a3afa256-b80c-4a6d-bd9c-786887c8ce4a" #e35c798a-cc47-4a32-955f-db52e9e7f57a" #"s3i:0a884197-1314-440b-920a-f1c63b46201d" #None
SECRET = "e35c798a-cc47-4a32-955f-db52e9e7f57a" #getpass.getpass('[S3I]: Please enter the secret:') 

In [23]:
IDP = s3i.IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)

In [24]:
ENDPOINT = "s3ibs://"+CLIENT #input("[S3I]: Please enter the endpoint for this inbox:")

In [25]:
class MessageEncryptor(pykka.ThreadingActor):
       
    def callback(self, pgp_message):
        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: A new message has been received")
        message_str = pgp_message.replace("\\n", "\n").strip('"') 
        uMsg = s3i.UserMessage(msg_blob=message_str)
    
        access_token = IDP.get_token(s3i.TokenType.ACCESS_TOKEN)
        dir = s3i.Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
        """Decrypt the message and verify the signature
        """
        if uMsg.pgpMsg.is_encrypted:
            sec_key = s3i.Key(path_demo_dir="key/", filename="secKey.asc")
            uMsg.decryptAndVerify(sec_key, "", dir)
            #uMsg.decrypt(sec_key.key)
            uMsg.convertPgpToMsg()

        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Subject of the message: " + uMsg.msg["subject"])
        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Text of the message:  " + uMsg.msg["text"])
        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Sender of the message:  :", dir.queryThingIDBased(uMsg.msg["sender"]+"/attributes/name"))

        attachments_list = uMsg.msg["attachments"]
        """
        store the attachment file in specified path
        """
        for attachment in attachments_list:
            with open("received_data/"+attachment["filename"], 'wb') as file:
                decode = base64.b64decode(attachment["data"])
                file.write(decode)
                print("[S3I]: Attachment " + attachment["filename"]
                   + " of the message is stored in received_data")
                            
        print("==============================================================================")

                            
messageEncryptor = MessageEncryptor.start().proxy()

In [26]:
receiver = receiver.Receiver.start(ENDPOINT,IDP,messageEncryptor).proxy()
receiver.start_receiving()

[S3I][2020-05-23 17:02:18]: A new message has been received
[S3I][2020-05-23 17:02:19]: The signature is valid.
[S3I][2020-05-23 17:02:19]: Decryption successful
[S3I][2020-05-23 17:02:19]: Subject of the message: tester
[S3I][2020-05-23 17:02:19]: Text of the message:  testertester
[S3I][2020-05-23 17:02:19]: Sender of the message:  : DT of gebhard_workshop
